<a href="https://colab.research.google.com/github/louistrue/learn-ifc-bfh25-D/blob/main/BFH-25-FireRating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/louistrue/learn-ifc-bfh25-D/blob/main/BFH-25-FireRating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# IFC-Workshop: FireRating finden und dokumentieren

## Lernziele
- FireRating-Informationen in Property Sets erkennen
- Informationen aus IFC mit IfcOpenShell automatisch auswerten.


In [1]:
%pip install ifcopenshell


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 MB 20.6 MB/s eta 0:00:00


## Theorie

### IFC-Hierarchie verstehen
- IFC organisiert Informationen hierarchisch: Projekt -> Standort -> Gebaeude -> Geschoss -> Bauteil.
- Nutzt die Struktur im Viewer, um relevante Elemente gezielt zu filtern und nicht den Kontext zu verlieren.

### Eigenschaften lesen
- Eigenschaften koennen als Schema-Attribute (z. B. `FireRating`) oder ueber Property Sets wie `Pset_DoorCommon` bzw. `Pset_WallCommon` kommen.
- In der Praxis tauchen unterschiedliche Schreibweisen auf: `FireRating`, `Brandklasse`, `Feuerwiderstand`, `EI30`, `T30`, `30 min`.
- Werte am Typ werden auf alle Vorkommen vererbt. Prueft immer Instanz **und** Typ, bevor ihr einen Wert als fehlend einstuft.

### Suche und Identifikation
- Jedes IFC-Element besitzt eine GUID; Viewer erlauben jedoch oft die Suche nach Name, Klasse oder Typ.

## Werkzeuge im Notebook
Mit den folgenden Zellen koennt ihr FireRating-Werte direkt in einer IFC-Datei nachweisen. Fuehrt sie Schritt fuer Schritt aus und passt Pfade sowie Klassen an euer Modell an.


In [ ]:
from pathlib import Path

def list_ifc_files(folder: str = ".") -> None:
    folder_path = Path(folder)
    if not folder_path.exists():
        print(f"Ordner {folder_path} nicht gefunden.")
        return
    files = sorted(p for p in folder_path.glob("*.ifc"))
    if not files:
        print("Keine IFC-Dateien im angegebenen Ordner gefunden. Bitte Pfad pruefen.")
        return
    print("Gefundene IFC-Dateien:")
    for file in files:
        size_kb = file.stat().st_size / 1024
        print(f"- {file.name} ({size_kb:.1f} KB)")

list_ifc_files()


### IFC-Datei laden
Passt `IFC_PATH` an eure Datei an. Wenn ihr dieses Notebook aus dem Team-Ordner startet, reicht oft der Dateiname.


In [ ]:
import ifcopenshell
from pathlib import Path

def load_ifc(path: str):
    file_path = Path(path)
    if not file_path.exists():
        print(f"Datei {file_path} nicht gefunden. Bitte Pfad pruefen.")
        return None
    model = ifcopenshell.open(str(file_path))
    try:
        entity_count = len(model)
    except TypeError:
        entity_count = sum(1 for _ in model)
    print(f"Geladen: {file_path.name} mit {entity_count} Entitaeten")
    return model

IFC_PATH = "small_house_20250918_212245.ifc"  # TODO: Pfad nach Bedarf anpassen
model = load_ifc(IFC_PATH)


### FireRating in Property Sets finden
Die folgenden Helferfunktionen durchsuchen Instanz- und Typ-Eigenschaften nach Begriffen, die auf FireRating hindeuten. Passe die Keywords bei Bedarf an eure Modellkonvention an.


In [ ]:
from typing import Dict, List, Optional

try:
    from ifcopenshell.util.element import get_psets
except ImportError as exc:  # pragma: no cover
    raise ImportError("ifcopenshell util.element nicht gefunden. Bitte Installation pruefen.") from exc

FIRE_KEYWORDS = ("firerating", "fire_rating", "fire resistance", "brand", "fireproof")


def _collect_matches(source, scope: str) -> List[Dict[str, str]]:
    matches: List[Dict[str, str]] = []
    if source is None:
        return matches
    value = getattr(source, "FireRating", None)
    if value not in (None, ""):
        matches.append({
            "scope": scope,
            "pset": "Attribut",
            "property": "FireRating",
            "value": str(value),
        })
    try:
        psets = get_psets(source, include_quantities=False)
    except TypeError:
        psets = get_psets(source)
    for pset_name, properties in psets.items():
        if not isinstance(properties, dict):
            continue
        for prop_name, prop_value in properties.items():
            key = prop_name.lower()
            if any(term in key for term in FIRE_KEYWORDS):
                matches.append({
                    "scope": scope,
                    "pset": pset_name,
                    "property": prop_name,
                    "value": str(prop_value),
                })
    return matches


def find_fire_rating(element) -> List[Dict[str, str]]:
    matches: List[Dict[str, str]] = []
    matches.extend(_collect_matches(element, "Instanz"))
    typed_by = getattr(element, "IsTypedBy", None)
    if typed_by:
        for rel in typed_by:
            type_obj = getattr(rel, "RelatingType", None)
            type_name = getattr(type_obj, "Name", "") if type_obj else ""
            scope = "Typ" if not type_name else f"Typ {type_name}"
            matches.extend(_collect_matches(type_obj, scope))
    return matches


def summarize_fire_rating(model, ifc_class: str = "IfcDoor") -> List[Dict[str, object]]:
    if model is None:
        return []
    rows: List[Dict[str, object]] = []
    for element in model.by_type(ifc_class):
        matches = find_fire_rating(element)
        primary = matches[0] if matches else None
        rows.append({
            "class": ifc_class,
            "global_id": element.GlobalId,
            "name": element.Name or "",
            "fire_rating": primary["value"] if primary else "",
            "source": f"{primary['scope']} / {primary['pset']}" if primary else "",
            "matches": matches,
        })
    return rows


def show_fire_rating(rows: List[Dict[str, object]], show_empty: bool = False, limit: Optional[int] = 20) -> None:
    if not rows:
        print("Keine Elemente gefunden oder IFC-Datei nicht geladen.")
        return
    data = rows if show_empty else [row for row in rows if row["fire_rating"]]
    if not data:
        print("Keine FireRating-Werte gefunden (nutze show_empty=True fuer eine vollstaendige Liste).")
        return
    limit = limit or len(data)
    print(f"Anzahl Elemente: {len(data)} (Anzeige max. {limit})")
    header = f"{'#':<3} {'Name':<30} {'GlobalId':<22} {'FireRating':<12} Quelle"
    print(header)
    print("-" * len(header))
    for idx, row in enumerate(data[:limit], start=1):
        name = (row["name"] or "-")[:30]
        print(f"{idx:<3} {name:<30} {row['global_id']:<22} {row['fire_rating']:<12} {row['source']}")


def show_missing_fire_rating(rows: List[Dict[str, object]], limit: Optional[int] = 20) -> None:
    missing = [row for row in rows if not row["fire_rating"]]
    if not missing:
        print("Alle Elemente besitzen einen FireRating-Wert.")
        return
    limit = limit or len(missing)
    print(f"{len(missing)} Elemente ohne FireRating (Anzeige max. {limit})")
    for idx, row in enumerate(missing[:limit], start=1):
        name = (row["name"] or "-")[:30]
        print(f"{idx:<3} {name:<30} {row['global_id']}")


def inspect_matches(row: Dict[str, object]) -> None:
    print(f"Element: {row['name'] or '-'} ({row['global_id']})")
    if not row["matches"]:
        print("Keine FireRating-Treffer fuer dieses Element gespeichert.")
        return
    for match in row["matches"]:
        print(f"- {match['scope']} | {match['pset']} | {match['property']} -> {match['value']}")


### Auswertung starten
Fuehre die folgende Zelle aus, um eine schnelle Uebersicht fuer Tueren zu erhalten. Aendere den Klassennamen auf `IfcWall`, wenn du Waende analysieren moechtest.


In [ ]:
door_rows = summarize_fire_rating(model, "IfcDoor")
show_fire_rating(door_rows, show_empty=False, limit=20)
print()
show_missing_fire_rating(door_rows, limit=20)


### Detailanalyse einzelner Elemente
Wenn ihr ein bestimmtes Element dokumentieren moechtet, nutzt `inspect_matches` mit einem Eintrag aus der Liste `door_rows` (oder `wall_rows`).


In [ ]:
if door_rows:
    inspect_matches(door_rows[0])  # Index bei Bedarf anpassen
